In [3]:
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
#from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GATConv

In [5]:
from torch_geometric.datasets import EllipticBitcoinDataset
dataset = EllipticBitcoinDataset(root='./pytorch_input')
data=dataset[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#70/30 split, first 34 timestep, occurs at index 136265
newsplit =  torch.Tensor([True]*136265+[False]*67504)
data.train_mask = torch.logical_and(data.y <2,newsplit)
data.test_mask = torch.logical_and(data.y <2,torch.logical_not(newsplit))

In [7]:
dataset.num_classes

2

In [14]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads, dropout=0.6)
        # On the Pubmed dataset, use `heads` output heads in `conv2`.
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1,
                             concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

#num features, hidden, out, heads
model = GAT(dataset.num_features, 64, dataset.num_classes+1,
            3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
classweight = torch.FloatTensor([0.3,0.7,0])

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask],weight=classweight)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=-1)

    accs = []
    for mask in [data.train_mask, data.test_mask]:
        accs.append(int((pred[mask] == data.y[mask]).sum()) / int(mask.sum()))
    return accs


# best_val_acc = final_test_acc = 0
for epoch in range(1, 201):
    loss = train()
    train_acc, tmp_test_acc = test()
    # if val_acc > best_val_acc:
    #     best_val_acc = val_acc
    #     test_acc = tmp_test_acc
    if epoch % 20 == 0:
        print("Epoch=",epoch, "Loss=",loss, "Train=",train_acc, "Test=",tmp_test_acc)

Epoch= 20 Loss= 0.8929781913757324 Train= 0.8251488593028702 Test= 0.5760647870425915


In [ ]:
from sklearn.metrics import roc_curve, confusion_matrix
pred = model(data.x, data.edge_index).argmax(dim=-1)
print(confusion_matrix(data.y[data.test_mask],pred[data.test_mask]))

[[15490    97]
 [ 1060    23]]
